In [1]:
import requests
import sqlite3
import time
import concurrent.futures
from rx import Observable
import json
from rx import create
from flask import Flask, render_template, request
from timeit import default_timer as timer
import sqlalchemy as db

In [2]:
def sequential_call():
    response_list = []
    for i in range(1,6,1):
        req = requests.get("https://rickandmortyapi.com/api/character/"+str(i)).json()
        response_list.append(req)
    #print(response_list)
    return response_list

In [3]:
def parallel_call():

    with concurrent.futures.ThreadPoolExecutor() as executor:
        response_list = []
        for i in range (1,6,1):
            future = executor.submit(requests.get, "https://rickandmortyapi.com/api/character/"+str(i))
            return_value = future.result()
            req = json.loads(return_value.text)
            response_list.append(req)
            #print(y["name"])
        return response_list

In [4]:
def write_in_parallel(temp):
    sqliteConnection = sqlite3.connect('SQLite_Python.db')
    cursor = sqliteConnection.cursor()
    for i in range(0,len(temp),1):
        params = (temp[i]['name'],temp[i]['status'],temp[i]['gender'])
        cursor.execute("INSERT INTO rickandmorty(name, status, gender) VALUES (?, ?, ?)",params)
        sqliteConnection.commit()
    sqliteConnection.close()

In [5]:
def reactive_call(observer, scheduler):
    for i in range(1,6,1):
        req = requests.get("https://rickandmortyapi.com/api/character/"+str(i)).json()
        observer.on_next(req)
    observer.on_completed()



In [10]:
def parallel_reactive_call(observer, scheduler):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        
        for i in range(1,6,1):
            req = requests.get("https://rickandmortyapi.com/api/character/"+str(i)).json()
            executor.submit(observer.on_next,req)
        observer.on_completed()



In [153]:
def insert_into_db(response):
    sqliteConnection = sqlite3.connect('SQLite_Python.db', check_same_thread=False)
    cursor = sqliteConnection.cursor()
    
    #check if table exists or not
    sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS rickandmorty (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        status text NOT NULL,
                                        gender text NOT NULL
                                    ); """
    cursor.execute(sql_create_projects_table)
    return insert(response)

In [154]:
def insert(response):
    for i in range(0,len(response),1):
        params = (response[i]['name'],response[i]['status'],response[i]['gender'])
        cursor.execute("INSERT INTO rickandmorty(name, status, gender) VALUES (?, ?, ?)",params)
        sqliteConnection.commit()
        return True

In [35]:
def fetch_from_db():
    sqliteConnection = sqlite3.connect('SQLite_Python.db')

    cursor = sqliteConnection.cursor()
    cursor.execute("SELECT * FROM rickandmorty")

    rows = cursor.fetchall()

    for row in rows:
        print(row)
    sqliteConnection.commit()
    sqliteConnection.close()

In [44]:
# Deleting all records now
def delete_all():
    sqliteConnection = sqlite3.connect('SQLite_Python.db')
    cursor = sqliteConnection.cursor()

    sql_delete_query = """DELETE from rickandmorty """
    cursor.execute(sql_delete_query)

    sqliteConnection.commit()
    sqliteConnection.close()

# Calling API in Sequential 

In [6]:
app = Flask(__name__)


@app.route("/", methods=['GET', 'POST'])
def seq():
    text = [] 
    time = 0
    if request.method == 'POST':
        if request.form['one'] == 'get data from API':
            #pass 
            start = timer()

            global temp 
            temp = sequential_call()
            #print(temp)
            end = timer()
            time = round(end - start, 2)
            for i in range (0,len(temp),1):
                text.append(str(temp[i]['id']) +":  "+  temp[i]['name'] +",  "+ temp[i]['status'] +",  "+ temp[i]['gender'])
        
        elif request.form['one'] == 'insert data into database':
            #print(temp)
            if(len(temp) > 0):
                sqliteConnection = sqlite3.connect('SQLite_Python.db')
                cursor = sqliteConnection.cursor()

                #check if table exists or not
                sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS rickandmorty (
                                                    id integer PRIMARY KEY,
                                                    name text NOT NULL,
                                                    status text NOT NULL,
                                                    gender text NOT NULL
                                                ); """
                cursor.execute(sql_create_projects_table)
                sqliteConnection.commit()
                
                for i in range(0,len(temp),1):
                    params = (temp[i]['name'],temp[i]['status'],temp[i]['gender'])
                    cursor.execute("INSERT INTO rickandmorty(name, status, gender) VALUES (?, ?, ?)",params)
                    sqliteConnection.commit()
                sqliteConnection.close()
                #print(insert_into_db(temp))
                text = []
                print("data inserted")
                text.append("data inserted into database")
#             #pass # do something else
#             render_template('parallel.html', result = text)
            
        elif request.form['one'] == 'get data from database':
            sqliteConnection = sqlite3.connect('SQLite_Python.db')

            cursor = sqliteConnection.cursor()
            cursor.execute("SELECT * FROM rickandmorty")

            rows = cursor.fetchall()

            for row in rows:
                text.append(row)
            sqliteConnection.commit()
            sqliteConnection.close()
        else:
            pass # unknown
    elif request.method == 'GET':
        print("No Post Back Call")
    return render_template('login.html', result = text, time = time)  

if __name__ == "__main__":
    app.run(debug = True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2021 23:48:16] "GET / HTTP/1.1" 200 -


No Post Back Call


127.0.0.1 - - [14/Mar/2021 23:48:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2021 23:48:25] "POST / HTTP/1.1" 200 -


data inserted


127.0.0.1 - - [14/Mar/2021 23:48:27] "POST / HTTP/1.1" 200 -


# Calling API in Parallel

In [53]:
app = Flask(__name__)


@app.route("/", methods=['GET', 'POST'])
def parallel():
    text = [] 
    time = 0
    if request.method == 'POST':
        if request.form['one'] == 'get data from API':
            #pass 
            start = timer()

            global temp 
            temp = parallel_call()
            #print(temp)
            end = timer()
            time = round(end - start, 2)
            for i in range (0,len(temp),1):
                text.append(str(temp[i]['id']) +":  "+  temp[i]['name'] +",  "+ temp[i]['status'] +",  "+ temp[i]['gender'])
        
        elif request.form['one'] == 'insert data into database':
            #print(temp)
            if(len(temp) > 0):
                sqliteConnection = sqlite3.connect('SQLite_Python.db')
                cursor = sqliteConnection.cursor()

                #check if table exists or not
                sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS rickandmorty (
                                                    id integer PRIMARY KEY,
                                                    name text NOT NULL,
                                                    status text NOT NULL,
                                                    gender text NOT NULL
                                                ); """
                cursor.execute(sql_create_projects_table)
                sqliteConnection.commit()                
                
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    future = executor.submit(write_in_parallel, temp)
                sqliteConnection.commit()
                sqliteConnection.close()

                text = []
                print("data inserted")
                text.append("data inserted into database")
            
        elif request.form['one'] == 'get data from database':
            sqliteConnection = sqlite3.connect('SQLite_Python.db')

            cursor = sqliteConnection.cursor()
            cursor.execute("SELECT * FROM rickandmorty")

            rows = cursor.fetchall()

            for row in rows:
                text.append(row)
            sqliteConnection.commit()
            sqliteConnection.close()
        else:
            pass # unknown
    elif request.method == 'GET':
        print("No Post Back Call")
    return render_template('login.html', result = text, time = time)  

if __name__ == "__main__":
    app.run(debug = True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Mar/2021 01:38:01] "GET / HTTP/1.1" 200 -


No Post Back Call


127.0.0.1 - - [15/Mar/2021 01:38:07] "POST / HTTP/1.1" 200 -


# Calling API in Sequential Reactive Programming

In [54]:
app = Flask(__name__)


@app.route("/", methods=['GET', 'POST'])
def seq_rx():
    text = [] 
    time = 0
    if request.method == 'POST':
        if request.form['one'] == 'get data from API':
            start = timer()
            global temp
            temp = []
            source = create(reactive_call)
            source.subscribe(
                on_next = lambda req: temp.append(req),
                on_error = lambda e: print("Error Occurred: {0}".format(e)),
                on_completed = lambda: print("Done!"),
            )
            for i in range (0,len(temp),1):
                text.append(str(temp[i]['id']) +":  "+  temp[i]['name'] +",  "+ temp[i]['status'] +",  "+ temp[i]['gender'])
            end = timer()
            time = round(end - start, 2)
            
        elif request.form['one'] == 'insert data into database':
            if(len(temp) > 0):
                sqliteConnection = sqlite3.connect('SQLite_Python.db')
                cursor = sqliteConnection.cursor()

                #check if table exists or not
                sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS rickandmorty (
                                                    id integer PRIMARY KEY,
                                                    name text NOT NULL,
                                                    status text NOT NULL,
                                                    gender text NOT NULL
                                                ); """
                cursor.execute(sql_create_projects_table)
                sqliteConnection.commit()
                
                for i in range(0,len(temp),1):
                    params = (temp[i]['name'],temp[i]['status'],temp[i]['gender'])
                    cursor.execute("INSERT INTO rickandmorty(name, status, gender) VALUES (?, ?, ?)",params)
                    sqliteConnection.commit()
                sqliteConnection.close()
                #print(insert_into_db(temp))
                text = []
                print("data inserted")
                text.append("data inserted into database")
            
        elif request.form['one'] == 'get data from database':
            sqliteConnection = sqlite3.connect('SQLite_Python.db')

            cursor = sqliteConnection.cursor()
            cursor.execute("SELECT * FROM rickandmorty")

            rows = cursor.fetchall()

            for row in rows:
                text.append(row)
            sqliteConnection.commit()
            sqliteConnection.close()
        else:
            pass # unknown
    elif request.method == 'GET':
        print("No Post Back Call")
    return render_template('login.html', result = text, time = time)  

if __name__ == "__main__":
    app.run(debug = True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Mar/2021 01:38:36] "GET / HTTP/1.1" 200 -


No Post Back Call


127.0.0.1 - - [15/Mar/2021 01:38:42] "POST / HTTP/1.1" 200 -


Done!


127.0.0.1 - - [15/Mar/2021 01:38:45] "POST / HTTP/1.1" 200 -


data inserted


127.0.0.1 - - [15/Mar/2021 01:38:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 01:38:48] "POST / HTTP/1.1" 200 -


data inserted


127.0.0.1 - - [15/Mar/2021 01:38:50] "POST / HTTP/1.1" 200 -


# Calling API in Parallel Reactive programming

In [11]:
app = Flask(__name__)


@app.route("/", methods=['GET', 'POST'])
def parallel_rx():
    text = [] 
    time = 0
    if request.method == 'POST':
        if request.form['one'] == 'get data from API':
            start = timer()
            global temp
            temp = []
            source = create(parallel_reactive_call)
            source.subscribe(
                on_next = lambda req: temp.append(req),
                on_error = lambda e: print("Error Occurred: {0}".format(e)),
                on_completed = lambda: print("Done!"),
            )
            for i in range (0,len(temp),1):
                text.append(str(temp[i]['id']) +":  "+  temp[i]['name'] +",  "+ temp[i]['status'] +",  "+ temp[i]['gender'])
            end = timer()
            time = round(end - start, 2)
            
        elif request.form['one'] == 'insert data into database':
            if(len(temp) > 0):
                sqliteConnection = sqlite3.connect('SQLite_Python.db')
                cursor = sqliteConnection.cursor()

                #check if table exists or not
                sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS rickandmorty (
                                                    id integer PRIMARY KEY,
                                                    name text NOT NULL,
                                                    status text NOT NULL,
                                                    gender text NOT NULL
                                                ); """
                cursor.execute(sql_create_projects_table)
                sqliteConnection.commit()
                
                for i in range(0,len(temp),1):
                    params = (temp[i]['name'],temp[i]['status'],temp[i]['gender'])
                    cursor.execute("INSERT INTO rickandmorty(name, status, gender) VALUES (?, ?, ?)",params)
                    sqliteConnection.commit()
                sqliteConnection.close()
                #print(insert_into_db(temp))
                text = []
                print("data inserted")
                text.append("data inserted into database")
            
        elif request.form['one'] == 'get data from database':
            sqliteConnection = sqlite3.connect('SQLite_Python.db')

            cursor = sqliteConnection.cursor()
            cursor.execute("SELECT * FROM rickandmorty")

            rows = cursor.fetchall()

            for row in rows:
                text.append(row)
            sqliteConnection.commit()
            sqliteConnection.close()
        else:
            pass # unknown
    elif request.method == 'GET':
        print("No Post Back Call")
    return render_template('login.html', result = text, time = time)  

if __name__ == "__main__":
    app.run(debug = True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Mar/2021 16:56:55] "GET / HTTP/1.1" 200 -


No Post Back Call


127.0.0.1 - - [15/Mar/2021 16:56:58] "POST / HTTP/1.1" 200 -


Done!


127.0.0.1 - - [15/Mar/2021 16:57:01] "POST / HTTP/1.1" 200 -


data inserted


127.0.0.1 - - [15/Mar/2021 16:57:03] "POST / HTTP/1.1" 200 -


In [45]:
# delete_all()

# sqliteConnection = sqlite3.connect('SQLite_Python.db')
# cursor = sqliteConnection.cursor()

# temp = parallel_call()
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     future = executor.submit(write_in_parallel, temp)
# fetch_from_db()

(1, 'Rick Sanchez', 'Alive', 'Male')
(2, 'Morty Smith', 'Alive', 'Male')
(3, 'Summer Smith', 'Alive', 'Female')
(4, 'Beth Smith', 'Alive', 'Female')
(5, 'Jerry Smith', 'Alive', 'Male')
